In [1]:
from datasets import load_dataset

dataset = load_dataset("clue", "cmnli")

Dataset clue downloaded and prepared to /home/mymusise/.cache/huggingface/datasets/clue/cmnli/1.0.0/e508b66266ba417d60e89ed8b167699cb4b56d3a2ead29b5667907d08069dbfc. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00, 616.60it/s]


In [4]:
labels = set(dataset['train']['label'])

In [5]:
from transformers import AutoTokenizer


model_path = "/data2/CPM-distill/CPM-Generate-distill"

tokenizer = AutoTokenizer.from_pretrained(model_path)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'CpmTokenizer'. 
The class this function is called from is 'XLNetTokenizer'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
def token_map(examples):
    return tokenizer(examples["sentence1"] + examples["sentence2"], padding="max_length", max_length=512, truncation=True, add_special_tokens=False)

train_dataset = dataset['train'].map(token_map)
eval_dataset = dataset['validation'].map(token_map)

100%|██████████| 12241/12241 [00:06<00:00, 1989.36ex/s]


In [10]:
train_dataset = train_dataset.shuffle(seed=42)
eval_dataset = eval_dataset.shuffle(seed=42)

In [12]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=len(labels)+1)

Some weights of the model checkpoint at /data2/CPM-distill/CPM-Generate-distill were not used when initializing GPT2ForSequenceClassification: ['lm_head.weight']
- This IS expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at /data2/CPM-distill/CPM-Generate-distill and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
# ensure some model padding token id
model.config.pad_token_id = tokenizer.pad_token_id

In [14]:
import numpy as np
import evaluate


metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [15]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", per_device_train_batch_size=16)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)
print(f"before train: {trainer.evaluate()}")
trainer.train()
print(f"after train: {trainer.evaluate()}")

The following columns in the evaluation set  don't have a corresponding argument in `GPT2ForSequenceClassification.forward` and have been ignored: sentence1, idx, sentence2. If sentence1, idx, sentence2 are not expected by `GPT2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 12241
  Batch size = 8
100%|██████████| 1531/1531 [01:44<00:00, 14.72it/s]
The following columns in the training set  don't have a corresponding argument in `GPT2ForSequenceClassification.forward` and have been ignored: sentence1, idx, sentence2. If sentence1, idx, sentence2 are not expected by `GPT2ForSequenceClassification.forward`,  you can safely ignore this message.
/home/mymusise/.local/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True`

before train: {'eval_loss': 1.2480762004852295, 'eval_accuracy': 0.3397598235438281, 'eval_runtime': 104.0964, 'eval_samples_per_second': 117.593, 'eval_steps_per_second': 14.708}


  1%|          | 500/73461 [03:00<7:55:02,  2.56it/s]Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json


{'loss': 1.1339, 'learning_rate': 4.96596833694069e-05, 'epoch': 0.02}


Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
  1%|▏         | 1000/73461 [06:01<7:21:12,  2.74it/s]Saving model checkpoint to test_trainer/checkpoint-1000
Configuration saved in test_trainer/checkpoint-1000/config.json


...

{'loss': 0.8392, 'learning_rate': 2.2774669552551693e-05, 'epoch': 1.63}


Model weights saved in test_trainer/checkpoint-40000/pytorch_model.bin
 55%|█████▌    | 40500/73461 [3:56:03<3:07:12,  2.93it/s]Saving model checkpoint to test_trainer/checkpoint-40500
Configuration saved in test_trainer/checkpoint-40500/config.json


{'loss': 0.8246, 'learning_rate': 2.243435292195859e-05, 'epoch': 1.65}


Model weights saved in test_trainer/checkpoint-40500/pytorch_model.bin


RuntimeError: [enforce fail at inline_container.cc:300] . unexpected pos 92190016 vs 92189968